# Dataset: Completamiento y producción









![Baner_imagen](https://i.ibb.co/3rMLZJ7/oil.jpg)

---

**Carolina Piedrahita | Analista de datos | Data Scientist**
* Correo: cpiedrahita@unal.edu.co
* Linkedin: www.linkedin.com/in/carolina-piedrahita
* Github: https://github.com/CarolinaPiedrahita/DataScienceOilandGas
* Nro: +57 310 285 47 73


______________________
![Baner_imagen](https://i.ibb.co/nmHwdx0/CicloDC.jpg)


## 0.- Importación de librerías


In [ ]:
import pandas as pd # Manejo de data estructurada (Dataframe)
import numpy as np #Manejo de matrices
import matplotlib.pyplot as plt # Gráficas
import seaborn as sns  #Gráficas sencillas
import folium  #'''Spatial Visualizations'''
import folium.plugins #'''Spatial Visualizations'''
import cv2 # para leer imagenes

from sklearn.model_selection import train_test_split # Separar datos en Train(Entrenamiento) y Test(Prueba)

**Funciones generales para graficar**

In [ ]:
def analisisNumericas(df, variable):
  # fig, ax = plt.subplots(1, 2,figsize=(12, 4))
  # ax[0].hist(df[variable], bins=25)
  # ax[1].boxplot(df[variable],0, 'gD')
  # plt.show()

  print(" "*20,"Histograma"," "*20)
  df[variable].plot.hist(bins=25,figsize=(8,4),color="#CABDE9")
  plt.show()
  print("\n")
  print(" "*20,"Boxplot"," "*20)
  df[variable].plot.box(figsize=(8,4))
  plt.show()

def analisisCategoricas(df, variable):
  frecuencia_absoluta = pd.DataFrame(df[variable].astype('str').fillna('Nulo').value_counts(dropna = False))
  frecuencia_absoluta.columns = ['FREC_ABS'] 
  frecuencia_relativa = pd.DataFrame(df[variable].astype('str').fillna('Nulo').value_counts(normalize = True, dropna = False))
  frecuencia_relativa.columns = ['FREC_REL'] 
  tablaResumen = pd.concat([frecuencia_absoluta,frecuencia_relativa], axis = 1)
  display(tablaResumen)
  tablaResumen
  plt.bar(x = tablaResumen.index, height = tablaResumen['FREC_ABS'], color="#CABDE9")
  for i, txt in enumerate(tablaResumen['FREC_ABS']):
    plt.annotate(str(txt), (i, txt), fontsize = 12)
  plt.twinx()
  plt.plot(tablaResumen['FREC_REL'], linestyle='-', linewidth=2.0, color='red')
  plt.ylim(0, 1)
  for i, txt in enumerate(tablaResumen['FREC_REL']):
    plt.annotate(str(round(txt * 100, 1)) + '%', (i, txt), fontsize = 12, ha='center')
  plt.show()

## 1. Comprensión del negocio

Una empresa operadora de un campo africano desea a través de un estudio responder a las siguientes preguntas: 

*   ¿Cuál es el completamiento que dá tasas más altas de producción?
*   ¿Cuál es el completamiento que dá tasas más bajas de producción?
*   ¿Cuál es el completamiento que dá tasas más altas de inyección?
*   ¿Cuál es el completamiento que dá tasas más bajas de inyección?
*   ¿Cuáles son las variables que más afectan la producción?

Y ***adicional se busca generar un modelo para predecir el valor del caudal de petróleo*** según el tipo de yacimiento. 

## 2. Comprensión de los datos

**En esta etapa nos avocaremos a conocer todo lo referente a los datos con los que contamos**

Las características  son: 

*   Qo: Caudal oil
*   Qw: Caudal agua
*   Qg: Caudal de gas
*   Year: Año
*   Month: Mes
*   wellAPI: API del pozo
*   Wellname: Nombre del pozo
*   BHP: Presión de fondo
*   CompL: Completamiento
*   FlowDays: Días de flujo
*   Type: Tipo
*   TD: Profundidad verdadera
*   reservoir: yacimiento
*   long: longitud
*   lat: latitud










### 2.1.- Recopilación de los Datos


In [ ]:
# Lectura de la base de datos desde Github en internet y asignación a la variable df.
path = "https://raw.githubusercontent.com/CarolinaPiedrahita/DataScienceOilandGas/main/DatasetCompletamientoProducer.csv"
df = pd.read_csv(path)
 

In [ ]:
df.shape
# INTERPRETACION:
# 3220 observaciones | registros o filas
# 15 columnas

In [ ]:
df.info() #Acá podemos observar el tipo de dato y la cantidad de registros asociados a cada variable.

In [ ]:
df.head() # Acá podemos visualizar como esta formado nuestro dataset, y ejemplo de los primeros 10 datos que lo conforman. 

In [ ]:
target = 'Qo' # sera el target dado que se vamos a  modelar con pozos productores

In [ ]:
df[target].value_counts() #Se realiza un conteo por tipo de tasa de producción de petróleo


In [ ]:
df['reservoir'].value_counts()
#Se realiza un conteo por tipo de yacimiento


In [ ]:
df['CompL'].value_counts() #Se realiza un conteo por completamiento

### 2.2 Identificación y conteo de valores faltantes:

Voy aplicar el método `isnull()` y a utilizar la función de agregación `sum()` para identificar los valores faltantes por variable.

In [ ]:
datos_faltantes = df.isnull().sum()
datos_faltantes

De acuerdo a la información anterior no tenemos datos faltantes para ningún campo.


Por lo cuál, no es necesario hacer tratamiento de datos faltantes, para fines académicos se listan las opciones de tratamiento de datos faltantes: 

<ol>
    <li>Eliminar el dato entero<br>
        a. Eliminar la fila entera<br>
        b. Eliminar la columna entera
    </li>
    <li>Reemplazar el dato<br>
        a. Reemplazar por la media<br>
        b. Reemplazar por la moda<br>
        c. Reemplazar basados en otra función
    </li>
</ol>

En este caso tenemos cierta libertad para aplicar diferences métodos de llenado de datos faltantes; no obstante, algunos métodos pueden resultar más adecuados que otros depende del caso. Ejemplo:

Teniendo en cuenta las buenas práctias sólo debemos eliminar una columna sí la mayoría de entradas son vacías. En nuestro caso, ninguna de las columnas cumplen con este criterio como para ser eliminadas. 



In [ ]:
#aparencia de la tabla luego de procesarla
df.head()

### 2.3 Corrección de datos

<p>Cómo un último paso en el preprocesado de los datos, se pasa a  revisar de que todos los datos estén en el formato adecuado(int, float, text u otro).</p>

Podemos usar en Pandas
<p><b>.dtypes()</b>  para ver el tipo</p>
<p><b>.astype()</b> para cambiar el tipo</p>

In [ ]:
df.dtypes

En nuestro caso vamos definir nuestras variables categóricas como "string" (o cadena de caracteres es un tipo de dato que se utiliza para almacenar textos).

In [ ]:
df[["wellAPI"]] = df[["wellAPI"]].astype("str")
df[["Wellname"]] = df[["Wellname"]].astype("str")
df[["Type"]] = df[["Type"]].astype("str")
df[["reservoir"]] = df[["reservoir"]].astype("str")
df[["Year"]] = df[["Year"]].astype("str")
df[["Month"]] = df[["Month"]].astype("str")

In [ ]:
df.dtypes

Se puede observar que todas las columnas tienen el tipo correcto. 



### 2.4 Exploración de los Datos

El análisis exploratorio de datos (EDA del inglés Exploratory Data Analysis) se realiza con el fin de:

<ol>
  <li>Resumir las principales características del conjunto de datos.</li>
  <li>Entender mejor el conjunto de datos que se está tratando.</li>
  <li>Establecer relaciones entre variables.</li>
  <li>Determinar si una variables es un buen predictor de una variable blanco.</li>
  <li>Extraer las variables más importantes.</li>
</ol>

**Objetivo:** Determinar las características que más impactan en el valor de la tasa de producción para el caso de estudio.

Para abordar el análisis exploratorio de datos, es útil aplicar las siguientes técnicas:

<ol>
  <li> Estadística descriptiva</li>
  <li> Agrupamiento de datos</li>  
  <li> Correlación (Mapas de correlación)</li>
</ol>



Antes de ponernos en la tarea de construir un modelo que pretenda predecir alguna característica del conjunto de datos, es importante realizar una exploración de los datos. La **estadística descriptiva** nos permite explorar los datos con el propósito de hallar las características principales, mediante la implementación de gráficas y el cálculo de algunas variable estadísticas. Para este fín, en Python, podemos hacer uso de:

<ol>
  <li>Métodos: <code>.describe()</code>, el cuál permite obtener un resumen estadístico de las <b>variables numéricas</b> en <code>pandas</code>, o la función <code>.value_counts()</code>, la cual permite contabilizar variables categóricas .</li>
    <li>Box plots. </li>
    <li>Gráficos de dispersión. </li>
</ol>

In [ ]:
df.describe() # Descripción estadística de los variables del dataset: desviación estandar (std), minimos y maximos, percentiles 25%, 50%, 75%.

###Definición de variables categóricas y númericas.

In [ ]:
# Definimos las variables que trataremos como numéricas y como categóricas de nuestro dataset generado
varCategoricas = ['reservoir','Year','Month']
varNumericas = ['Qo', 'Qw','CompL','FlowDays','TD'] 

#### Sacamos el análisis categórico para cada variable.


In [ ]:
#Sacamos el analisis categorico para cada variable
for categorica in varCategoricas:
    print("-"*50,categorica,"-"*50)
    analisisCategoricas(df, categorica)
    plt.figure(figsize=(10,10))
    print("\n\n")

#### **Interpretación variables categóricas:**
* Se tienen 3220 datos para pozos productores.
* La mayoría de los yacimientos son tipo Red para este dataset se tienen el  59.7% en esta modalidad y el 23% tipo Maine.
* El año en el cuál se tiene más registros es el 2009 y 2013, los meses para los cuáles más registros se reportan en este dataset son Octubre (9.3% de los datos), Marzo (8.9%) y Septiembre (8.8%).

#### Sacamos el análisis númerico para cada variable.

**Interés del box-plot** El máximo interés del box-plot es visualizar la distribución de una variable numérica de la manera más simplificada posible. Sólo utiliza los valores de los cuartiles, los extremos (el mínimo y el máximo) y valores raros o outliers.

No depende de valores ponderados como la media. Simplemente se fija en las características de la posición. En el fondo, primero ordena la variable y después la caracteriza según la posición de los cuartiles y los valores extremos.

![Baner_imagen](https://i.ibb.co/NjZMXXb/box2.png)

________________________________
**Interés del histograma** Interpretar las variaciones de los datos.
Ofrecer una mejor visión de la información proporcionada por los datos para interpretarlos más adecuadamente.

![Baner_imagen](https://i.ibb.co/wW7cP8k/histograma.jpg) 

In [ ]:
for numerica in varNumericas:
  print("#"*30,numerica,"#"*30)
  analisisNumericas(df, numerica)
  plt.figure(figsize=(10,10))
  print("\n\n")

#### **Interpretación variables númericas**

**Comentario:** Del análisis de las variables numéricas se puede observar lo siguiente:
1. **Qo:** No posee una distribucion normal, posee outliers superiores.
2. **Qw:**  No posee una distribucion normal, posee outliers superiores. Histograma con islas separadas
3. **CompL:** Se observa que no sigue una distribución normal, histograma con islas separadas y posee outliers superiores.
4. **FlowDays:** No sigue una distribución normal, posee outliers inferiores.
5. **TD:**  No posee una distribucion normal, posee 2 outliers superiores.Histograma con islas separadas








### Tratamiento de Outlier y nulos en el target Qo y completamiento

En las gráficas anteriores (boxplot), vemos que entre Q1 y Q3 sabemos que están el 50% de los valores obtenidos en el estudio. A esta distancia se le llama rango intercuantílico (IQR: InterQuantile Range).

Para el tratamiento de estos outlier se decide definir una función que reciba como parámetros el conjunto de datos, el umbral inferior y el umbral superior. Y en la función se decide reemplazar por la media aquellos valores atípicos que estén por debajo del umbral inferior, y por la mediana aquellos que estén por encima del umbral superior. Cómo se muestra a continuación:



In [ ]:
# ## Eliminando los outliers de Qo
#target = 'Qo'
## Eliminando los outliers del Qo
Q1_target = df[target].quantile(0.25)
Q3_target = df[target].quantile(0.75)
#se calcula el rango intercuantílico IQR
IQR_target = Q3_target - Q1_target
#Se define como valor atípico leve 
#aquel que dista 1,5 veces el rango intercuantílico por debajo de Q1 o por encima de Q3
inf_target= Q1_target - 1.5 * IQR_target #vamos a calcular cual sería el umbral por abajo
sup_target= Q3_target + 1.5 * IQR_target #vamos a calcular cual sería el umbral por arriba
print('inf_target:',inf_target, 'sup_target:',sup_target)
#Quitando outlier
df=df[(df[target] >= inf_target) & (df[target] <= sup_target)]
print('Tamano despues de filtar los outliers de target: Qo',df.shape)
##Eliminando nulos
df.dropna(inplace=True)
print('Tamano despues de filtar los nulos de target',df.shape)

In [ ]:
CompL ='CompL' #declarando la variable
# ## Eliminando los outliers de CompL 
Q1_CompL = df[CompL].quantile(0.25)
Q3_CompL  = df[CompL].quantile(0.75)
#se calcula el rango intercuantílico IQR
IQR_CompL  = Q3_CompL - Q1_CompL 
#Se define como valor atípico leve 
#aquel que dista 1,5 veces el rango intercuantílico por debajo de Q1 o por encima de Q3
inf_CompL = Q1_CompL  - 1.5 * IQR_CompL  #vamos a calcular cual sería el umbral por abajo
sup_CompL = Q3_CompL  + 1.5 * IQR_CompL  #vamos a calcular cual sería el umbral por arriba

print('inf_CompL :',inf_CompL , 'sup_CompL :',sup_CompL )
#Quitando outlier
df=df[(df[CompL] >= inf_CompL) & (df[CompL] <= sup_CompL)]
print('Tamano despues de filtar los outliers de CompL',df.shape)

##Eliminando nulos
df.dropna(inplace=True)
print('Tamano despues de filtar los nulos de CompL',df.shape)

###Mapa de referencia de la ubicación donde se concentra la mayor cantidad de información seguú coordenadas registradas.

In [ ]:
m = folium.Map(location=[-30.559482, 22.937506], zoom_start=6, tiles="Stamen Terrain")

folium.Marker(
    location=[-33.14892407, 21.22751324],
    popup="reservoir1",
    icon=folium.Icon(icon="info-sign"),
).add_to(m)

folium.Marker(
    location=[-33.21392663, 21.21755058],
    popup="reservoir2",
    icon=folium.Icon(color="green"),
).add_to(m)

folium.Marker(
    location=[-33.21621517, 21.24374328],
    popup="reservoir3",
    icon=folium.Icon(color="red", icon="info-sign"),
).add_to(m)


m

### Correlacción de variables con respecto a Qo, Qw

In [ ]:
# Eliminamos columnas que no nos aportan información relevante para la correlación
df = df.drop(['BHP'], axis=1) 
df = df.drop(['lat'], axis=1) 
df = df.drop(['long'], axis=1) 



In [ ]:
#Generamos un gráfico de calor para ver la relación porcentual que existe
#Entre cada variable 
plt.figure(figsize=(10,10))
sns_plot= sns.heatmap(df.corr(method ='pearson'), annot=True, fmt='.0%')

Observamos la varianza a través de múltiples variables, revelando cualquier patrón, mostrando si las variables son similares entre sí y para detectar si existen correlaciones entre ellas.

**¿Cuáles son las variables que más afectan la producción?**

Se observa que **los atributos más influyentes en la producción de petróleo** que cuenta con un 26% (Qw), 21% (Qg), 16%(FlowDays), 11% (CompL) de ocurrencias respecto al atributo Qo, influyen de manera directamente proporcional en la producción de petróleo. A mayor producción de agua, mayor producción de gas, mayor completamiento (pies) mayor producción de petróleo.


**¿Cuáles son las variables que más afectan la tasa de producción de agua?**

Se observa que **los atributos más influyente en la producción de agua** que cuenta con un 29% (completamiento) de ocurrencias respecto al atributo Qw, influye directamente proporcional en la producción de agua. Igual a mayor completamiento (pies) mayor producción de agua.

Y se oberva una correlación pequeña -5% inversamente proporcional entre Qw y 
la profundidad verdadera (TD). A menor profundidad verdadera mayor producción de agua, se refiere entonces a que llega más agua en pozos inclindados.


### **Veamos gráficamente cómo se comportan estas variables más influyentes con respecto a la tasa de producción de petróleo y agua.**

### Comportamiento de la tasa de producción de petróleo con respecto a la tasa de producción de agua.

In [ ]:
df[['Qo','Qw']].plot(figsize=(16,9))
plt.show()

### Comportamiento de la tasa de producción con respecto al completamiento.

In [ ]:
df[['Qo','CompL']].plot(figsize=(16,9))
plt.show()

### Completamiento vs. tasa de producción de petróleo.

In [ ]:
g= sns.set(style='whitegrid')
fig, ax = plt.subplots(figsize=(80,20))
sns.boxplot(x=df.CompL,y=df.Qo, width=.9)
plt.title("Completamiento vs. tasa de producción de petróleo", fontsize=80)
#plt.ylim(0, 2)  # limitar y, para que los 'outliers' no ocupan la mayoría del espacio
sns.despine(top=True,
            right=True,
            left=True,
            bottom=False)

**¿Cuál es el completamiento que dá tasas más altas de producción de petróleo?**

El completamiento que dá tasas más altas de producción de petróleo es el de 36,08924 (ft), su caja naranja se ve más arriba que los demás y observa que este alcanza tasas de produccion de petróleo máximas de hasta 62 (bbl/d). 


**¿Cuál es el completamiento que dá tasas más bajas de producción de petróleo?**

El completamiento que dá tasas más bajas de producción es el de 95.14436 (ft)su caja es prácticamente una línea se ve más abajo que los demás y observa que este alcanza tasas de producción mínimas de hasta 9 (bbl/d); esto puede deberse a otros factor (valores atípicos, etc) que se sugiere explorar detenidamente para contrastar la información de manera adecuada.


In [ ]:
g= sns.set(style='whitegrid')
fig, ax = plt.subplots(figsize=(80,20))
sns.boxplot(x=df.CompL,y=df.Qw, width=.9)
plt.title("Completamiento vs. tasa de producción de agua", fontsize=80)
#plt.ylim(0, 2)  # limitar y, para que los 'outliers' no ocupan la mayoría del espacio
sns.despine(top=True,
            right=True,
            left=True,
            bottom=False)

**¿Cuál es el completamiento que dá tasas más altas de producción de agua?**

El completamiento que dá tasas más altas de producción de agua es el de 95,80735 (ft), su caja morada se vé más arriba que los demás y observa que este alcanza tasas de producción de agua máximas de hasta 2000 (bbl/d). Esto justifica que su producción de petróleo sea la mínima). Revisar si se conecta con formaciones de agua (acuífero) cercanas. 

**¿Cuál es el completamiento que dá tasas más bajas de producción de agua?**
El completamiento que dá tasas más bajas de producción de agua es el de 75,07141 (ft), su caja azul agua marina se vé más abajo que los demás y se observa que este alcanza tasas de producción de agua mínimas de hasta 10 (bbl/d). 


## 3. Modelamiento

## 3.1. Regresión lineal

Este modelo pretende predecir la tasa de producción dependiendo del tipo de yacimiento. 

In [ ]:
# Eliminamos columnas que no nos aportan información relevante para el modelo
df = df.drop(['Type'], axis=1) #wellAPI no nos dice nada acerca del Qo
df = df.drop(['wellAPI'], axis=1) #wellAPI no nos dice nada acerca del Qo
df = df.drop(['Wellname'], axis=1) #wellAPI no nos dice nada acerca del Qo

In [ ]:
X_lineal=df.drop(['reservoir'], axis=1).values
y_lineal=df[target].values

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

# Seleccionamos los datos de entrenamiento y prueba
X_lineal_train, X_lineal_test, y_lineal_train, y_lineal_test = train_test_split(X_lineal,y_lineal,test_size=0.3,random_state=3)



In [ ]:
# Entrenamos el modelo
linear  = LinearRegression(normalize=True)
linear.fit(X_lineal_train,y_lineal_train)

Obtengamos ahora los datos predichos por el modelo y calculemos las métricas.

In [ ]:
y_lineal_pred = linear.predict(X_lineal_test)

In [ ]:
print('MAE: ', metrics.mean_absolute_error(y_lineal_test, y_lineal_pred))
print('MSE: ', metrics.mean_squared_error(y_lineal_test, y_lineal_pred))
print('RMSE: ', np.sqrt(metrics.mean_squared_error(y_lineal_test, y_lineal_pred)))

MAE:  5.574956911235456e-15
MSE:  4.0132915858630674e-29
RMSE:  6.335054526886937e-15


Y calculemos el coeficiente de determinación R2.

In [ ]:
print('R2: ', metrics.r2_score(y_lineal_test, y_lineal_pred))

R2:  1.0


Se comprueba además si los errores se distribuyen según una distribución normal, lo que nos da una prueba de la valídez de nuestro modelo. Mediante un histograma de residuos.

In [ ]:
sns.distplot((y_lineal_test - y_lineal_pred), bins = 50)
plt.show()

Una vez se entrena el modelo de regresión lineal, las predicciones se obtienen rápidamente.


## 3.2. RandomForestRegressor




Este modelo pretende predecir la tasa de producción de  dependiendo del tipo de yacimiento. 

In [ ]:
X=df.drop([target,'reservoir'], axis=1).values
y=df[target].values

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [ ]:
for rs in range(3): 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=rs)

    ## Haciendo el barrido de parámetros
    parameters = {'max_depth': np.arange(1,40,1)}
    estimator=GridSearchCV(RandomForestRegressor(n_estimators=100,random_state=rs), parameters, cv=5,n_jobs=-1)
    estimator.fit(X,y)
    print(estimator.best_params_)

    ## Entrenando y evaluado el modelo
    reg=RandomForestRegressor(n_estimators=100,max_depth=estimator.best_params_['max_depth'],random_state=rs)
    reg.fit(X_train,y_train)
    y_pred=reg.predict(X_test)
    print ('r2 ','tTest:',r2_score(y_test,y_pred), 'tTrain:',r2_score(y_train,reg.predict(X_train)))
    print ('MAE ','tTest:',mean_absolute_error(y_test,y_pred), 'tTrain:',mean_absolute_error(y_train,reg.predict(X_train)))
    print ('RMSE ','tTest:',np.sqrt(mean_squared_error(y_test,y_pred)), 'tTrain:',np.sqrt(mean_squared_error(y_train,reg.predict(X_train))))
    print ('stdTest:',np.std(y_test),'stdTrain:',np.std(y_train),'stdAll:',np.std(y))
    print ('ErrorTest:', np.sqrt(mean_squared_error(y_test,y_pred))/np.mean(y_test), 
           'ErrorTrain:',np.sqrt(mean_squared_error(y_train,reg.predict(X_train)))/np.mean(y_train),
           'ErrorAll:', np.sqrt(mean_squared_error(y_test,y_pred))/np.mean(y))
    ## Graficando el las carateristicas principales
    ejex=range(X.shape[1])
    ejey=reg.feature_importances_
    plt.figure()
    plt.bar(ejex,ejey)
    plt.xticks(ejex,list(df.drop(target,axis=1).columns),rotation=90)
    plt.show()
    plt.close()

**Conclusión**

La técnica que mostró mejor desempeño de forma general es el RANDOMFOREST, mostrando un buen r2= 0.68 de entrenamiento para profundidades máximas superiores a 31. Adicional se decide elegir Randomforest en vez de regresión lineal teniendo en cuenta la no linealidad de los datos. En este caso, Randomforest puede hacer un mejor trabajo al capturar la no linealidad en los datos dividiendo el espacio en subespacios más pequeños dependiendo de las preguntas formuladas y aportar un resultado más preciso de el valor de Qo.